In [1]:
import numpy as np
import pandas as pd

# viz
import matplotlib.pyplot as plt

# notebook settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 1000)

## Sample Prep

In [2]:
samples = pd.read_csv('../data/TCGA/rna-seq_pan/meta/gdc_sample_sheet.2019-12-12.tsv', sep="\t")
# get file type
samples['data'] = [val[1] for i,val in samples['File Name'].str.split(".").items()]
samples['project'] = [val[1] for i,val in samples['Project ID'].str.split("-").items()]
samples['project'].value_counts()

BRCA    1206
LUAD     588
UCEC     567
KIRC     554
LUSC     543
LGG      524
PRAD     517
COAD     506
THCA     505
SKCM     469
BLCA     431
LIHC     421
STAD     402
OV       379
KIRP     308
CESC     306
PAAD     171
ESCA     171
GBM      166
TGCT     150
PCPG     133
LAML     123
KICH      81
ACC       75
CHOL      41
SARC      10
DLBC       9
READ       7
MESO       1
Name: project, dtype: int64

In [3]:
samples['Sample Type'].value_counts()

Primary Tumor                                      8166
Solid Tissue Normal                                 636
Metastatic                                          385
Primary Blood Derived Cancer - Peripheral Blood     123
Recurrent Tumor                                      43
Additional - New Primary                             10
Additional Metastatic                                 1
Name: Sample Type, dtype: int64

## New Model based on Tissues with available metastatic samples

In [4]:
samples[samples['Sample Type']=='Metastatic']['project'].value_counts()
samples[samples['Sample Type']=='Primary Tumor']['project'].value_counts().head(9)

SKCM    364
BRCA      7
THCA      7
CESC      2
COAD      1
PCPG      1
PAAD      1
PRAD      1
ESCA      1
Name: project, dtype: int64

BRCA    1087
UCEC     544
LUAD     528
LGG      506
LUSC     496
KIRC     483
PRAD     466
COAD     465
THCA     445
Name: project, dtype: int64

In [5]:
proj = np.append(samples['project'].value_counts().head(9).index.values, ['SKCM'])

cases = samples[samples['Sample Type']=='Primary Tumor'].sample(frac=1).copy()
cases.shape
cases = cases[cases['project'].isin(proj)]
cases['project'].value_counts()
cases.shape

(8166, 10)

BRCA    1087
UCEC     544
LUAD     528
LGG      506
LUSC     496
KIRC     483
PRAD     466
COAD     465
THCA     445
SKCM     103
Name: project, dtype: int64

(5123, 10)

## Dataset Prep

In [6]:
from sklearn.model_selection import train_test_split

target = 'project'
cases[target] = cases[target].astype('category')
train, test = train_test_split(cases)

In [7]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit
import visualization as vis
import numpy as np

if torch.cuda.is_available():
    cuda = torch.cuda.is_available()
    print("{} GPUs available".format(torch.cuda.device_count()))

classes = train[target].cat.categories.values


8 GPUs available


In [8]:
from tcga_datasets import TCGA, SiameseTCGA
root_dir = "../data/TCGA/rna-seq_pan/"
batch_size = 1

train_dataset = TCGA(root_dir, samples=train, train=True, target=target)
test_dataset = TCGA(root_dir, samples=test, train=False, target=target)
print('Loaded')
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

Loaded


## Siamese Network

In [23]:
# Step 1 set up dataloader
siamese_train_dataset = SiameseTCGA(train_dataset) # Returns pairs of images and target same/different
siamese_test_dataset = SiameseTCGA(test_dataset)
batch_size = 64
kwargs = {'num_workers': 20, 'pin_memory': True} if cuda else {}
siamese_train_loader = torch.utils.data.DataLoader(siamese_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
siamese_test_loader = torch.utils.data.DataLoader(siamese_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from tcga_networks import EmbeddingNet, SiameseNet
from losses import ContrastiveLoss
from metrics import AccumulatedAccuracyMetric

# Step 2
embedding_net = EmbeddingNet()
# Step 3
model = SiameseNet(embedding_net)
if cuda:
    model = nn.DataParallel(model)
    model.cuda()
    
# Step 4
margin = 1.
loss_fn = ContrastiveLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 8
# print training metrics every log_interval * batch_size
log_interval = 4

DataParallel(
  (module): SiameseNet(
    (embedding_net): EmbeddingNet(
      (fc): Sequential(
        (0): Linear(in_features=60483, out_features=2000, bias=True)
        (1): PReLU(num_parameters=1)
        (2): Linear(in_features=2000, out_features=500, bias=True)
        (3): PReLU(num_parameters=1)
        (4): Linear(in_features=500, out_features=250, bias=True)
        (5): PReLU(num_parameters=1)
        (6): Linear(in_features=250, out_features=100, bias=True)
        (7): PReLU(num_parameters=1)
        (8): Linear(in_features=100, out_features=50, bias=True)
        (9): PReLU(num_parameters=1)
        (10): Linear(in_features=50, out_features=10, bias=True)
        (11): PReLU(num_parameters=1)
        (12): Linear(in_features=10, out_features=2, bias=True)
      )
    )
  )
)

In [ ]:
train_loss, val_loss = fit(siamese_train_loader, siamese_test_loader, model, loss_fn, optimizer, scheduler, 
    n_epochs, cuda, log_interval)

Train: [0/3842 (0%)]	Loss: 0.154764
Train: [256/3842 (7%)]	Loss: 5.473493
Train: [512/3842 (13%)]	Loss: 0.770664
Train: [768/3842 (20%)]	Loss: 3.586725
Train: [1024/3842 (26%)]	Loss: 0.429813
Train: [1280/3842 (33%)]	Loss: 1.186276
Train: [1536/3842 (39%)]	Loss: 1.381489
Train: [1792/3842 (46%)]	Loss: 0.410460
Train: [2048/3842 (52%)]	Loss: 1.311575
Train: [2304/3842 (59%)]	Loss: 0.689415
Train: [2560/3842 (66%)]	Loss: 1.807112
Train: [2816/3842 (72%)]	Loss: 0.843931
Train: [3072/3842 (79%)]	Loss: 0.441966
Train: [3328/3842 (85%)]	Loss: 0.202272
Train: [3584/3842 (92%)]	Loss: 0.217853
Train: [120/3842 (98%)]	Loss: 0.938776
Epoch: 1/8. Train set: Average loss: 1.2938
Epoch: 1/8. Validation set: Average loss: 4.2793
Train: [0/3842 (0%)]	Loss: 11.663042
Train: [256/3842 (7%)]	Loss: 0.767174
Train: [512/3842 (13%)]	Loss: 0.702323
Train: [768/3842 (20%)]	Loss: 13.336773
Train: [1024/3842 (26%)]	Loss: 2.797228
Train: [1280/3842 (33%)]	Loss: 1.121683
Train: [1536/3842 (39%)]	Loss: 0.719549
Tr

In [ ]:
plt.plot(range(0, n_epochs), train_loss, 'rx-')
plt.plot(range(0, n_epochs), val_loss, 'bx-')

In [ ]:
classes = train[target].cat.categories.values
train_embeddings_cl, train_labels_cl = vis.extract_embeddings(train_loader, model)
vis.plot_embeddings(train_embeddings_cl, train_labels_cl, classes)

In [ ]:
val_embeddings_baseline, val_labels_baseline = vis.extract_embeddings(test_loader, model)
vis.plot_embeddings(val_embeddings_baseline, val_labels_baseline, classes)

### Project Metastatic SKCM onto learned space

In [ ]:
skcm_cat = np.where(cases['project'].cat.categories.values=='SKCM')[0][0]

In [ ]:
ms = samples[(samples['Sample Type']=='Metastatic') & (samples['project']=='SKCM')].sample(frac=1).copy()
ms[target] = [i + '-MET' for i in ms[target]]
ms[target] = ms[target].astype('category')

met_classes = ms[target].cat.categories.values

In [ ]:
root_dir = "../data/TCGA/rna-seq_pan/"
batch_size = 1

ms_dataset = TCGA(root_dir, samples=ms, train=False, target=target)
print('Loaded')
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
ms_loader = torch.utils.data.DataLoader(ms_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [ ]:

ms_embeddings_baseline, ms_labels_baseline = vis.extract_embeddings(ms_loader, model)

In [ ]:
comb_classes = np.append(classes, met_classes)

comb_embeddings = np.append(train_embeddings_cl, ms_embeddings_baseline, axis=0)
comb_embeddings.shape

ms_labels = np.repeat(np.unique(train_labels_cl).max() + 1, len(ms_labels_baseline))
comb_labels = np.append(train_labels_cl, ms_labels, axis=0)
comb_labels.shape

In [ ]:
vis.plot_embeddings(comb_embeddings, comb_labels, comb_classes)